In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from matplotlib.collections import PolyCollection
from matplotlib.animation import FuncAnimation

In [ ]:
def process_file(file_name, sort_option = 'none'):
    
    df = pd.read_csv(file_name, delimiter='|', header=None, skiprows=1)
    Q = []
    seeds = []
    polygons = []
    data = []

    for index, row in df.iterrows():
        
        # get seedpoints
        seed_data = row[0]
        seed_points = np.array(seed_data.split(','), dtype=float)

        # get polygons for cells
        coords_data = row[1]
        poly = []
        for item in coords_data.split(';'):
            if item:
                points = item.split(",")
                points = np.array(points, dtype=float)
                poly.append(points)
        poly.append(poly[0])

        # get Q value or list of quantities
        q_data = row[2]
        if ',' in str(q_data):
            q_split = np.array(q_data.split(','), dtype=float)
        else:
            q_split = float(q_data)

        data.append((seed_points, poly, q_split))

    # sort data by x or y direction if needed
    if sort_option == 'x':
        data.sort(key=lambda x: x[0][0])
    elif sort_option == 'y':
        data.sort(key=lambda x: x[0][1])

    seeds, polygons, Q = zip(*data)

    return np.array(seeds), list(polygons), np.array(Q)

### 2D-image

In [ ]:
seed, polygons, Q = process_file('files/cmesh1990.csv')

# plot mesh with Q
fig, ax = plt.subplots(figsize = (12, 10))
norm = mcolors.Normalize(vmin=0, vmax=1)
collection = PolyCollection(polygons, array=Q, cmap='viridis', norm=norm, edgecolor='face') # edgecolor = 'none' / 'face'
ax.add_collection(collection)
ax.autoscale_view()
cbar = fig.colorbar(collection, ax=ax)
cbar.set_label('Q-value')
plt.savefig('figures/image1.png')
plt.show()

### 2D-animation

In [ ]:
# define figure and axes
fig, ax = plt.subplots(figsize=(12, 10))
norm = mcolors.Normalize(vmin=0, vmax=1)
collection = PolyCollection([], array=[], cmap='viridis', norm=norm)
collection.set_edgecolor('face') # edgecolor = 'none' / 'face'
ax.add_collection(collection)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
#ax.set_title("Conway's Game of Life - Voronoi")
cbar = fig.colorbar(collection, ax=ax)
cbar.set_label('Q-value')


# animation update function
def update(frame):
    file_name = f"files/cmesh{frame}.csv"
    seeds, polygons, Q = process_file(file_name)
    collection.set_paths(polygons)
    collection.set_array(Q)
    return collection,

# animation
frames = range(0, 500, 1)
fps = 10
ani = FuncAnimation(fig, update, frames=frames, blit=True, repeat=False)
ani.save('figures/animation_conway_test.gif', fps=fps)
plt.show()


### 1D-plot

In [ ]:
# 1D-plot of vmesh and cartesian Q values
#seed_v, polygons_v, Q_v = process_file("files/vmesh199.csv", sort_option='y')
seed_c, polygons_c, Q_c = process_file("files/cmesh1990.csv", sort_option='x')



#plt.scatter(seed_v[:, 1], Q_v, marker = '.', label = 'vmesh')
plt.scatter(seed_c[:, 0], Q_c, marker = '.', label = 'cartesian')
plt.title("vmesh vs cartesian for 2k cells 20sec, 200step")
plt.xlabel("y")
plt.ylabel("Q-value")
plt.ylim(0, 1)
plt.legend()
plt.show()

In [ ]:
# functions to either plot 1D-animation of Q over time or binned 1D-animation of Q on 2D mesh over time

def process_and_plot(file_path, sort_option, bin_size = 45):
    seed, polygons, Q = process_file(file_path, sort_option=sort_option)
    #avg_seed = [np.mean(seed[i:i+bin_size, 0]) for i in range(0, int(len(seed[:, 0])), bin_size)]
    #avg_Q = [np.mean(Q[i:i+bin_size]) for i in range(0, int(len(Q)), bin_size)]
    return seed[:, 0], Q, #avg_seed, avg_Q

def analytic_Q(x, frame, dt, velocity):

    dx = frame * dt * velocity

    if (x > dx and x<0.1+dx):
        return 1
    else:
        return 0

v = 0.5
dt = 0.00003

def calc_L1_error(y_c, Q_c, frame):

    y_c = np.array(y_c)
    Q_c = np.array(Q_c)

    Q_analytic = np.array([analytic_Q(y, frame, dt, v) for y in y_c])

    L1 = (1/len(Q_c)) * np.sum(np.abs(Q_c - Q_analytic))

    return L1

lsp = np.linspace(0, 1, 10000)
L1s = []

def update(frame):
    plt.cla()
    #y_v, Q_v = process_and_plot(f"files/vmesh{frame}.csv", 'y')#, avg_y_v, avg_Q_v = process_and_plot(f"files/vmesh{frame}.csv", 'y')
    y_c, Q_c = process_and_plot(f"files/cmesh{frame}.csv", 'x')#, avg_y_c, avg_Q_c = process_and_plot(f"files/cmesh{frame}.csv", 'y')
    #plt.scatter(y_v, Q_v, marker='.', label = 'vmesh', color = 'cornflowerblue')
    #plt.plot(avg_y_v, avg_Q_v, color = 'tab:blue', label = 'binned vmesh')
    plt.plot(y_c, Q_c, marker='.', label = 'cartesian', color = 'sandybrown')
    #plt.plot(avg_y_c, avg_Q_c, color = 'tab:orange', label = 'binned cartesian')

    Q_analytic = [analytic_Q(x, frame, dt, v) for x in lsp]
    L1 = calc_L1_error(y_c, Q_c, frame)
    L1s.append(L1)

    plt.plot(lsp, Q_analytic, color = 'black', label = 'analytic')
    plt.title(f"advection upwind, cartesian1D")
    plt.xlabel("x")
    plt.ylabel("Q-value")
    plt.ylim(0, 1)
    plt.legend()

fig, ax = plt.subplots()
frames = range(0, 20000, 100)
ani = FuncAnimation(fig, update, frames=frames, repeat=True)
ani.save('figures/1d_animation.gif', fps=30)
plt.show()


In [ ]:
#frames_2000step_100grid = frames
#L1s_20000step_200grid = L1s


#plt.plot(L1s_20000step_200grid, label = '2xgrid, 10x time')
#plt.plot(L1s_2000step_100grid, label = 'grid, time')
plt.title('L1 error increase over time')
plt.xlabel('simulation step')
plt.ylabel('L1 error')
plt.legend()
plt.savefig('figures/L1_error_time_evolution.png')
plt.show()

In [ ]:
# plot difference in total Q over simulation steps
df = pd.read_csv('files/total_Q_diff.csv', decimal=',', header=None)
diff_Q = np.nan_to_num(df.astype(float).to_numpy().flatten())

plt.plot(-diff_Q[:-1])

plt.yscale('log')
#plt.xlim(min(diff_Q[:-1]), max(diff_Q[:-1])+0.0000001)

plt.title("Change in total Q over the Mesh - Vmesh advection")
plt.xlabel('simulation step')
plt.ylabel('total_Q - total_Q_initial ')
plt.savefig("figures/delta_Q_total.png")
plt.show()

print(len(diff_Q[:-1]))